In [59]:
#!/usr/bin/env python3
import numpy as np
import pandas as pd
import random
import time
import datetime
import re
import tensorflow as tf
import torch 
from torch.utils.data import DataLoader, Dataset, TensorDataset,random_split, RandomSampler, SequentialSampler, IterableDataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import classification_report


device = 'cuda' if torch.cuda.is_available() else 'cpu'


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

def report_average(report_list):
    r_list = list()
    for report in report_list:
        splited = [' '.join(x.split()) for x in report.split('\n\n')]
        header = [x for x in splited[0].split(' ')]
        data = np.array(splited[1].split(' ')).reshape(-1, len(header) + 1)
        data = np.delete(data, 0, 1).astype(float)
        df = pd.DataFrame(data, columns=header)
        r_list.append(df)
    tmp = pd.DataFrame()
    for df in r_list:
        tmp = tmp.add(df, fill_value=0)           
    report_ave =  tmp/len(r_list)
    return(report_ave)

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_fscore(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat, average='macro')

#In [1]: import torch
#In [2]: import torch.nn.functional as F 
#In [3]: probs =  F.softmax(logits_tensor)
#probs = probs.detach().cpu().numpy()
#predictions  = np.argmax(probs, axis=1).flatten()

###matthews_corrcoef(labels, preds)


class CustomDataset(Dataset):
    def __init__(self,filename,name):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        if name == 'covid':
            self.df = pd.read_csv(filename,delimiter='\t',encoding='utf-8')  
            #self.train_df = pd.read_csv("/home/joao/COVID19Tweet-master/train.tsv",delimiter='\t',encoding='utf-8')  
            #self.val_df = pd.read_csv("/home/joao/COVID19Tweet-master/valid.tsv",delimiter='\t',encoding='utf-8')   
            self.df = self.df.rename(columns={'Text': 'sentence'})
            self.df = self.df.rename(columns={'Label': 'label'})
            self.df['label'].replace('INFORMATIVE', 1)
            self.df['label'] = self.df['label'].replace('INFORMATIVE', 1)
            self.df['label'].replace('UNINFORMATIVE', 0)
            self.df['label'] = self.df['label'].replace('UNINFORMATIVE', 0)
        if name == 'crisislext26':
            self.df = pd.read_csv("/home/joao/crisisLexT26.csv", encoding='utf-8')
            self.df = self.df.drop(['Tweet ID', ' Information Source', ' Information Type' ], axis=1)
            #Relabelling the columns titles to remove white spaces
            self.df = self.df.rename(columns={' Tweet Text': 'sentence'})
            self.df = self.df.rename(columns={' Informativeness': 'label'})
            self.df = self.df[self.df.label!= 'Not related']
            self.df = self.df[self.df.label!= 'Not applicable']
            self.df['label'].replace('Related and informative', 1)
            self.df['label'] = self.df['label'].replace('Related and informative', 1)
            self.df['label'].replace('Related - but not informative', 0)
            self.df['label'] = self.df['label'].replace('Related - but not informative', 0)
            self.df = self.df.reset_index(drop=True)
        if name == 'crisislext6':
            #self.df = pd.read_csv("/home/renato/Datasets/CrisisLexT6-v1.0/CrisisLexT6/2012_Sandy_Hurricane/2012_Sandy_Hurricane-ontopic_offtopic.csv", encoding='utf-8')
            #self.df = pd.read_csv("/home/joao/2012_Sandy_Hurricane-ontopic_offtopic.csv",encoding='utf-8')
            self.df = pd.read_csv("/home/joao/crisisLexT6.csv", encoding='utf-8')
            self.df = self.df.rename(columns={' tweet': 'sentence'})
            self.df = self.df.rename(columns={' label': 'label'})
            self.df['label'].replace('on-topic', 1)
            self.df['label'] = self.df['label'].replace('on-topic', 1)
            self.df['label'].replace('off-topic', 0)
            self.df['label'] = self.df['label'].replace('off-topic', 0)
        self.df = self.df[['sentence','label']]
        self.df['nchars'] = self.df['sentence'].str.len()
        self.df['nwords'] = self.df['sentence'].str.split().str.len()
        self.df['bhash'] = self.df["sentence"].str.contains(pat = '#',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['nhash'] = self.df["sentence"].str.count('#') 
        self.df['blink']  = self.df["sentence"].str.contains(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE, regex = True) .astype(int)
        self.df['nlink'] = self.df["sentence"].str.count(pat = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', flags=re.IGNORECASE) 
        self.df['bat'] = self.df["sentence"].str.contains(pat = '@',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['nat'] = self.df["sentence"].str.count(pat = '@') 
        self.df['rt'] = self.df["sentence"].str.contains(pat = '@rt|rt@',flags=re.IGNORECASE, regex = True).astype(int) 
        #df['phone'] = df["sentence"].str.contains(pat = '\(?([0-9]{3})\)?([ .-]?)([0-9]{3})\2([0-9]{4})',flags=re.IGNORECASE, regex = True).astype(int) 
        self.df['dlex'] = self.df["sentence"].apply(self.lexical_diversity)
        self.df["sentence"] = self.df["sentence"].str.lower()
        ## List of  US slangs.
        slangList = ['ASAP','BBIAB','BBL','BBS','BF','BFF','BFFL','BRB','CYA','DS','FAQ','FB','FITBLR','FLBP','FML','FTFY','FTW','FYI','G2G','GF','GR8','GTFO','HBIC','HML','HRU','HTH','IDK','IGHT','IMO','IMHO','IMY','IRL','ISTG','JK','JMHO','KTHX','L8R','LMAO','LMFAO','LMK','LOL','MWF','NM','NOOB','NP','NSFW','OOAK','OFC','OMG','ORLY','OTOH','RN','ROFL','RUH','SFW','SOML','SOZ','STFU','TFTI','TIL','TMI','TTFN','TTYL','TWSS','U','W/','WB','W/O','WYD','WTH','WTF','WYM','WYSIWYG','Y','YMMV','YW','YWA']
        slangList = [x.lower() for x in slangList]
        #happy emojis
        happy_emojis = [':\)', ';\)', '\(:']
        #sad emojis
        sad_emojis = [':\(', ';\(', '\):']
        punctuation = ['.',',','...','?','!',':',';']    
        #','-','+','*','_','=','/','','%',' &','{','}','[',']','(',')','
        #Checks if the sentence contains slang
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(slangList)))
        df1 = self.df[~mask]
        self.df['slang'] = mask.astype(int) 
        #Checks if the sentence contains happy emojis
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(happy_emojis)), regex = True)
        df1 = self.df[~mask]
        self.df['hemojis'] = mask.astype(int) 
        #Checks if the sentence contains happy emojis
        mask = self.df.iloc[:, 0].str.contains(r'\b(?:{})\b'.format('|'.join(sad_emojis)), regex = True)
        df1 = self.df[~mask]
        self.df['semojis'] = mask.astype(int) 
        self.hand_features =  self.df[['nchars', 'nwords','bhash','nhash','blink','nlink','bat','nat','rt','slang','dlex']]
        self.hand_features_DF = pd.DataFrame(self.hand_features)
        #################
        self.df['sentence'] = self.df['sentence'].str.replace(r'http(\S)+', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'http(\S)+', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'http ...', r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'@[\S]+',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'_[\S]?',r'')
        self.df['sentence'] = self.df['sentence'].str.replace(r'[ ]{2, }',r' ')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&amp;?',r'and')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&lt;',r'<')
        self.df['sentence'] = self.df['sentence'].str.replace(r'&gt;',r'>')
        self.df['sentence'] = self.df['sentence'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')
        self.df['sentence'] = self.df['sentence'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')
        self.df['sentence'] = self.df['sentence'].str.lower()
        self.df['sentence'] = self.df['sentence'].str.strip()
        self.sentences = self.df['sentence']
        self.labels = self.df['label'].values
        self.maxlen = 0
        if name == 'covid':
            self.maxlen = 512
        else:
            for sent in self.sentences:
                input_ids = self.tokenizer.encode(sent, add_special_tokens=True)
                self.maxlen = max(self.maxlen, len(input_ids))
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        sentence = self.df.loc[idx, 'sentence']
        label = self.df.loc[idx, 'label']
        h_features = self.hand_features_DF.loc[idx,['nchars', 'nwords','bhash','nhash','blink','nlink','bat','nat','rt','slang','dlex']]
        h_features_tensor = torch.tensor(h_features).to(device)
        tokens = self.tokenizer.tokenize(sentence)
        encoded_dict = self.tokenizer.encode_plus(tokens, add_special_tokens = True, max_length = self.maxlen, pad_to_max_length = True,return_attention_mask = True)
        tokens_ids = encoded_dict['input_ids']
        tokens_ids_tensor = torch.tensor(tokens_ids).to(device) #Converting the list to a pytorch tensor
        attn_mask = encoded_dict['attention_mask']
        attn_mask_tensor = torch.tensor(attn_mask).to(device)
        label_tensor = torch.tensor(label).to(device)
        return tokens_ids_tensor, attn_mask_tensor, label_tensor,h_features_tensor
    def lexical_diversity(self,text):
        return len(set(text.split())) / len(text.split())

def train(model, train_dataloader, validation_dataloader=None, epochs=4, evaluation=False):
    training_stats = []
    total_t0 = time.time()
    for epoch_i in range(0, epochs):  # For each epoch...
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        t0 = time.time() ###Measure how long the training epoch takes.
        total_train_loss = 0 ### Reset the total loss for this epoch.
        model.train()   ### Put the model into training mode.
        for step, batch in enumerate(train_dataloader):   ### For each batch of training data...
            if step % 40 == 0 and not step == 0: ### Progress update every 40 batches.
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            b_input_ids = batch[0].to(device)             ### `batch` contains three pytorch tensors:    #   [0]: input ids      #   [1]: attention masks    #   [2]: labels
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()  ### Clear any previously calculated gradients before performing a backward pass.
            loss, logits, hidden_states = model(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)   #### Perform a forward pass
            total_train_loss += loss.item() ### Accumulate the training loss over all of the batches
            loss.backward() ### Perform a backward pass to calculate the gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) ### Clip the norm of the gradients to 1.0 to help prevent the "exploding gradients" problem.
            optimizer.step()  ### Update parameters and take a step using the computed gradient.
            scheduler.step()
        avg_train_loss = total_train_loss / len(train_dataloader) ### Calculate the average loss over all of the batches.
        training_time = format_time(time.time() - t0)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))
        print("")
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            print("Running Validation...")
            model.eval() ### Put the model in evaluation mode--the dropout layers behave differently during evaluation.
            total_eval_accuracy = 0
            total_eval_fscore = 0
            total_eval_loss = 0
            nb_eval_steps = 0
            report_list  = []    
            for batch in val_dataloader:  # Unpack this training batch from our dataloader.
                b_input_ids = batch[0].to(device)
                b_input_mask = batch[1].to(device)
                b_labels = batch[2].to(device)
                with torch.no_grad():  ### Tell pytorch not to bother with constructing the compute graph during  the forward pass, since this is only needed for backprop (training).
                    (loss, logits, hidden_states) = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask,labels=b_labels)
                total_eval_loss += loss.item() ### Accumulate the validation loss.
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                #total_eval_accuracy += flat_accuracy(logits, label_ids) ###  Calculate the accuracy for this batch of test sentences, and accumulate it over all batches.
                pred_flat = np.argmax(logits, axis=1).flatten()
                labels_flat = label_ids.flatten()
                report = classification_report(labels_flat, pred_flat)
                report_list.append(report)
            #avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
            #print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
            avg_val_loss = total_eval_loss / len(val_dataloader)
            validation_time = format_time(time.time() - t0) ### Measure how long the validation run took.
            print("  Validation Loss: {0:.2f}".format(avg_val_loss))
            print("  Validation took: {:}".format(validation_time))
            print()
            print(report_average(report_list))
            print()
            training_stats.append(
                {
                    'epoch': epoch_i + 1,
                    'Training Loss': avg_train_loss,
                    'Valid. Loss': avg_val_loss,
                    'Training Time': training_time,
                    'Validation Time': validation_time
                }
            )
    print("Training complete!")
    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))  


train_dataset =  CustomDataset("/home/joao/COVID19Tweet-master/train.tsv","covid")
val_dataset =  CustomDataset("/home/joao/COVID19Tweet-master/valid.tsv","covid")


dataset =  CustomDataset(None,"crisislext6")

dataset =  CustomDataset(None,"crisislext26")


# Create a 90-10 train-validation split.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))



batch_size = 8

train_dataloader = DataLoader(train_dataset,sampler = SequentialSampler(train_dataset), batch_size = batch_size )
val_dataloader = DataLoader(val_dataset,sampler = SequentialSampler(val_dataset), batch_size = batch_size )

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels = 2,output_attentions = False, output_hidden_states = True)
model.to(device)

optimizer = AdamW(model.parameters(),lr = 2e-5,eps = 1e-8 )

epochs = 2
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,num_training_steps = total_steps)

set_seed(42)

train(model, train_dataloader, val_dataloader, epochs=4, evaluation=True)


# Display floats with two decimal places.
pd.set_option('precision', 3)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
df_stats = df_stats.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats


In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()


from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
  pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
  matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
  matthews_set.append(matthews)

# Create a barplot showing the MCC score for each batch of test samples.
ax = sns.barplot(x=list(range(len(matthews_set))), y=matthews_set, ci=None)

plt.title('MCC Score per Batch')
plt.ylabel('MCC Score (-1 to +1)')
plt.xlabel('Batch #')

plt.show()